In [110]:
import numpy as np
#import pyplot as plt
import scipy.constants as const

In [111]:
# définition de toutes les constantes utiles 
pi = const.pi
R=10
N=1000
E = 1
h = const.h
hbar = const.hbar
M = const.proton_mass
alpha = 1
beta = 1
nu = E / h
omega = 2*pi*nu

Soit H l’Hamiltonien d’un oscillateur anharmonique


$H = −\frac{\hbar^2}{2M} \frac{d^2}{dx^2}+ V (x)$ $(4.6)$ 


où le potentiel anharmonique s’écrit 


$V(x) = \frac{1}{2} M \omega^2x^2 + \alpha x^3 + \beta x^4$. $(4.7)$ \\


Au lieu de solutionner un système aux valeurs propres sur une base de
fonctions, on discrétisera l’espace sur une grille unidimensionnelle formée
de $N$ points ${x_j}$ telle que 


$x_1 = −R$ 

$x_2 = −R + h$ 

$... = ...$ 

$x_j = −R + (j − 1)h$ $(4.8)$ 

$... = ...$ 

$x_N = −R + (N − 1)h = +R$ 


avec $h = \frac{2x}{(N − 1)}$ et $R$ choisi suffisamment grand pour que $\psi_n(±R) ' 0$ où $H\psi_n(x) = E_n \psi_n(x)$.

On commence par discrétisé l'opérateur différentielle sur la grille unidimensionelle.

$\frac{d^2f_i}{dx^2} \approx \frac{f_{i-1}-2f_i + f_{i+1}}{\Delta x^2}$ 

On peut adapter l'équation ci-dessus pour l'ensemble de l'espace discrétisé en utilisant la matrice suivante :

$\begin{pmatrix}
-2 & 1 & 0 & 0 & \dot \\
1 & -2 & 1 & \dot & 0 \\
0 & 1 & \dot & 1 & 1 \\
0 & \dot & 1 & -2 & 1 \\
\dot & 0 & 1 & 1 & -2 \\
\end{pmatrix}$



In [112]:
def discrétise_espace(R, N):
    return list(-R + (i-1) * (2*R)/(N-1) for i in range(1, N+1))

In [113]:
def differentielle_discretiser(N):
    D = np.zeros((N, N))
    D[0, 0] = 2
    D[N-1, N-1] = 2
    D[0, 1] = -1
    D[N-1, N-2] = -1

    for i in range(1, N-1):
        D[i, i-1] = -1
        D[i, i+1] = -1
        D[i, i] = 2
        
    return D

In [114]:
def x_carré(R, N):
    return np.array([element**2 for element in discrétise_espace(R, N)])

def x_cube(R, N): 
    return np.array([element**3 for element in discrétise_espace(R, N)])

def x_quatre(R, N): 
    return np.array([element**4 for element in discrétise_espace(R, N)])

Il y a une erreur avec la fonction V !

In [115]:
def V(R, N, alpha, beta, M, omega):
    return (1/2)*M*discrétise_espace(R, N)*omega**2 + alpha*x_cube(R, N) + beta*x_quatre(R, N)

In [116]:
print(V(7, 15, 0, 0, 1, 1))

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
def H(R, N, alpha, beta, M, omega):
    return (hbar**2/(2*M))*differentielle_discretiser(N) + np.diag(V(R, N, alpha, beta, M, omega))